# Imports

In [ ]:
!!pip install litellm
import os
from google.colab import userdata

In [ ]:
from openai import OpenAI
from typing import List, Dict
import sys
import json
import re

# Handle Interactions

This function handles user interactions with the LLM using ChatML, with each messaging expected to have a system, user, or assistant.

In [ ]:
def generate_response(messages: List[Dict]) -> str:
   """Call LLM to get response using Deepseek."""
   # Retrieve Deepseek API key from Colab secrets and set environment variable
   deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
   client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

   response = client.chat.completions.create(
      model="deepseek-reasoner", # Specify the Deepseek model
      messages=messages,
      max_tokens=1024,
      stream=False
   )
   return response.choices[0].message.content

def list_files() -> List[str]:
    """List all files in the current directory."""
    return [f for f in os.listdir('.') if os.path.isfile(f)]

def read_file(file_name: str) -> str:
    """Read the content of a file."""
    with open(file_name, 'r') as f:
        return f.read()

In [ ]:
goals = """
You are an AI agent that specializes in the political science subfield of political psychology.

You have a similar level of knowledge and analytical skills as a PhD in Political Psychology graduate.

You are specifically an expert in political behavior, understanding political motivations, with an interest in how these impact both local and global geopolitical structures.

At least one external source must be used to support each bullet point.

Keep bullet point answers to 5 bullet points (or less) with up to 100 words that best summarize a quality answer.

Keep sentence answers to a maximum of 250 words total, no matter the complexity of the question.
"""

actions = """
If a user asks a question, extract the main points, use inductive reasoning to generalize to modern and historical examples.

Then, use deductive reasoning to answer the question.

Then, cite each source using a link to the source.
"""

information = """
Be mindful of any items in the memory and make sure that the logic follows in subsequent outputs.
"""

language = """
Respond in this format:

```
Question: <question>

Bullet Point Answer:
- <bullet point 1>
- <bullet point 2>
- ...
- <bullet point n >= 5>

Citations:
- <citation 1>
- <citation 2>
- ...
- <citation n>

Paragraph Answer:
<Paragraph answer>
```
"""

In [ ]:
agent_rules = [{
    "role": "system",
    "content": f"""
{goals}

{actions}

{information}

{language}
"""}]


In [ ]:
memory = [
    {"role": "system", "content": "Keep responses within 400 words total."}
]

In [ ]:
print("Hello, I am a political science AI agent imbued with knowledge about political psychology and behavior.")

# The Agent Loop
while True:
    user_input = input("Please enter your inquiry (type 'stop' to end interaction): ")

    if user_input.lower() == "stop":
        print("Stopping agent interaction.")
        break

    # Add user's new query to memory
    memory.append({"role": "user", "content": user_input})

    # 1. Construct prompt: Combine agent rules with current memory.
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")
    print()

    # 3. The LLM's response directly adheres to the agent_rules format.

    # 4. Update memory with the LLM's response
    memory.append(
        {"role": "assistant", "content": response}
    )


Hello, I am a political science AI agent imbued with knowledge about political psychology and behavior.
Please enter your inquiry (type 'stop' to end interaction): What are the negative impacts of nationalism in the tri-party affairs of China, South Korea, and Japan?
Agent thinking...
Agent response: ```
Question: What are the negative impacts of nationalism in the tri-party affairs of China, South Korea, and Japan?

Bullet Point Answer:
- Nationalism fuels historical grievances, blocking reconciliation over Japan's wartime past and complicating diplomatic relations with both China and South Korea, as seen in recurring disputes over "comfort women" and textbook narratives.
- It exacerbates territorial disputes, such as over the Diaoyu/Senkaku Islands and Dokdo/Takeshima, increasing military patrols and risks of escalation while hindering joint resource management.
- Economic relations suffer from nationalist-driven boycotts and trade restrictions, exemplified by Chinese consumer boycot